In [78]:
import cv2
import numpy as  np
import matplotlib.pyplot as plt
import json
from itertools import permutations
import random
import os
import copy

In [ ]:
def conc_two_images(image_1, image_2):
    height = min(image_1.shape[0], image_2.shape[0])
    width1 = int(image_1.shape[1] * height / image_1.shape[0])
    width2 = int(image_2.shape[1] * height / image_2.shape[0])

    # Resize images
    image1_resized = cv2.resize(image_1, (width1, height))
    image2_resized = cv2.resize(image_2, (width2, height))

    border_width = 10  # Width of the border in pixels
    border_color = [255, 255, 255]  # Border color, white in this case
    border = np.full((height, border_width, 3), border_color, dtype=np.uint8)

    # Concatenate the images horizontally
    concatenated_image = np.hstack((image1_resized, border, image2_resized))
    return concatenated_image

def flip_horizontal(image):
    flipped_image = cv2.flip(image, 1)
    return flipped_image

    
def flip_vetical(image):
    flipped_image = cv2.flip(image, 0)
    return flipped_image

def add_noise(image, mean = 25, var= 4000):
    row, col, ch = image.shape
    sigma = var ** 0.5
    gauss = np.random.normal(mean, sigma, (row, col, ch))
    gauss = gauss.reshape(row, col, ch)

    noisy_image = (image + gauss)

    # # Convert back to uint8
    noisy_image = np.clip(noisy_image, 0, 255).astype(np.uint8)
    return noisy_image

def rotate_img(image, min= -150, max= 150):
    angle = random.randint(min,max)
    height, width = image.shape[:2]
    center = (width // 2, height // 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0) 
    rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height))
    return angle, rotated_image


In [89]:
idx_to_aug_mapping = {
    1: "gaussian",
    2: "rotation",
    3: "vertical_flip",
    4: "horizontal_flip"
}
options = [1,2,3,4]
def list_non_empty_subsets(options):
    subsets = []
    for r in range(1, len(options)+1):
        for combo in permutations(options, r):
            if len(combo) > 1:
                subsets.append(combo)
    return subsets
non_empty_subsets = list_non_empty_subsets(options)
random.shuffle(non_empty_subsets)


In [90]:
# choosing 4 random permutations:
def augs():
    chosen = []
    while len(chosen) < 4:
        idx = random.randint(0,59)
        if idx in chosen:
            continue
        else:
            chosen.append(idx)
    return chosen

In [144]:
# Load an image
labels= {
    "id" : 0,
    "image_name" : "",
    "comparison": False,
    "original_image_name": None,
    "augmentations": {
        "gaussian_noise": False,
        "rotation_angle": 0,
        "vertical_flip": False,
        "horizontal_flip": False
    },
    "question": "",
    "gt_answer": "",
    "model_answer": ""
}
json_list = []
path = "new_test\\check"
images = [f for f in os.listdir(path) if f.endswith(".jpeg") or f.endswith(".jpg")]
for filename in images:
    print(filename, "FILE")
    image_dict = copy.deepcopy(labels)
    image_dict["image_name"] = filename
    json_list.append(image_dict)
    original_image_name = os.path.splitext(os.path.basename(filename))[0]
    # print(original_image_name)
    file_path = os.path.join(path, filename)
    image = cv2.imread(file_path)
    chosen = augs()
    # print(chosen, "values")
    for count, idx in enumerate(chosen):
        image_dict = copy.deepcopy(labels)
        image_dict["original_image_name"] = filename
        image_name = original_image_name
        augmentations = non_empty_subsets[idx]
        # print(augmentations)
        to_change = image
        for augmentation in augmentations:
            if augmentation == 1:
                aug_image = add_noise(to_change)
                image_name += "_gaussian"
                image_dict["augmentations"]["gaussian_noise"] = True
            elif augmentation == 2:
                angle, aug_image = rotate_img(to_change)
                # print(angle)
                image_name+= f"_{angle}_rotation"
                image_dict["augmentations"]["rotation_angle"] = angle
            elif augmentation == 3:
                aug_image = flip_vetical(to_change)
                image_name += "_fv"
                image_dict["augmentations"]["vertical_flip"] = True
            elif augmentation == 4:
                aug_image = flip_horizontal(to_change)
                image_name += "_fh"
                image_dict["augmentations"]["horizontal_flip"] = True
            
            to_change = aug_image
        # print(image_name, "hi")
        # print(count, "count")
        if count == 3 and original_image_name.split('_')[0] != "comparison":
            to_change = conc_two_images(image, to_change)
            image_name += "_comparison"
            image_dict["comparison"] = True
        
        image_name += ".jpeg"
        image_name = os.path.join(path, image_name)
        image_dict["image_name"] = image_name
        json_list.append(image_dict)
        cv2.imwrite(image_name, to_change)


comparison_1.jpg FILE
comparison_2.jpeg FILE
comparison_3.jpg FILE
comparison_4.jpg FILE
mmhal_1.jpg FILE
mmhal_2.jpg FILE
mmhal_3.jpg FILE
mmhal_4.jpg FILE
pinterest_1.jpeg FILE
pinterest_2.jpeg FILE
pinterest_3.jpeg FILE
pinterest_5.jpeg FILE
pinterest_6.jpeg FILE
pinterest_7.jpeg FILE
unsplash_1.jpg FILE
unsplash_2.jpg FILE
unsplash_3.jpg FILE


In [ ]:
with open("new_responses.json",  'w') as f:
    json.dump(json_list, f, indent = 4)